<a href="https://colab.research.google.com/github/2303A52144/ExplainableAI_Assignment/blob/main/ExplainableAI_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem - 1

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

data = pd.DataFrame({
    "Webinars_Attended": [0, 1, 2, 0, 1],
    "Sign_ups": [5, 15, 25, 8, 18]
})

X = data[["Webinars_Attended"]]
y = data["Sign_ups"]

model = LinearRegression()
model.fit(X, y)

slope = model.coef_[0]
intercept = model.intercept_

baseline = y.mean()

data["Predicted"] = model.predict(X)

data["SHAP_Value"] = data["Predicted"] - baseline

data["Baseline+SHAP"] = baseline + data["SHAP_Value"]

data["Prediction_Comparison"] = np.where(data["Predicted"] > y, "Over Prediction",
                                         np.where(data["Predicted"] < y, "Under Prediction", "Exact"))

print("Linear Regression Coefficients:")
print(f"Intercept: {intercept}")
print(f"Slope: {slope}")
print("\nBaseline:", baseline)
print("\nTable of SHAP values and predictions:")
print(data)


Linear Regression Coefficients:
Intercept: 6.714285714285715
Slope: 9.357142857142854

Baseline: 14.2

Table of SHAP values and predictions:
   Webinars_Attended  Sign_ups  Predicted  SHAP_Value  Baseline+SHAP  \
0                  0         5   6.714286   -7.485714       6.714286   
1                  1        15  16.071429    1.871429      16.071429   
2                  2        25  25.428571   11.228571      25.428571   
3                  0         8   6.714286   -7.485714       6.714286   
4                  1        18  16.071429    1.871429      16.071429   

  Prediction_Comparison  
0       Over Prediction  
1       Over Prediction  
2       Over Prediction  
3      Under Prediction  
4      Under Prediction  


# Problem - 2

In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

data = pd.DataFrame({
    "Webinars": [3, 2, 1, 4, 2],
    "Blogs": [5, 3, 4, 2, 1],
    "Signups": [60, 45, 40, 55, 35]
})

X = data[["Webinars", "Blogs"]]
y = data["Signups"]

model = LinearRegression()
model.fit(X, y)

intercept = model.intercept_
coef_webinars = model.coef_[0]
coef_blogs = model.coef_[1]

baseline = y.mean()

data["Predicted"] = model.predict(X)

data["SHAP_Webinars"] = coef_webinars * (data["Webinars"] - X["Webinars"].mean())
data["SHAP_Blogs"] = coef_blogs * (data["Blogs"] - X["Blogs"].mean())

data["Baseline+SHAP"] = baseline + data["SHAP_Webinars"] + data["SHAP_Blogs"]

data["Prediction_Comparison"] = np.where(data["Predicted"] > y, "Over Prediction",
                                         np.where(data["Predicted"] < y, "Under Prediction", "Exact"))

print("Intercept:", intercept)
print("Coefficient for Webinars:", coef_webinars)
print("Coefficient for Blogs:", coef_blogs)
print("Baseline:", baseline)
print(data)


Intercept: 15.58823529411766
Coefficient for Webinars: 7.745098039215684
Coefficient for Blogs: 4.274509803921567
Baseline: 47.0
   Webinars  Blogs  Signups  Predicted  SHAP_Webinars  SHAP_Blogs  \
0         3      5       60  60.196078       4.647059     8.54902   
1         2      3       45  43.901961      -3.098039     0.00000   
2         1      4       40  40.431373     -10.843137     4.27451   
3         4      2       55  55.117647      12.392157    -4.27451   
4         2      1       35  35.352941      -3.098039    -8.54902   

   Baseline+SHAP Prediction_Comparison  
0      60.196078       Over Prediction  
1      43.901961      Under Prediction  
2      40.431373       Over Prediction  
3      55.117647       Over Prediction  
4      35.352941       Over Prediction  


# Problem - 3

In [4]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

data = load_diabetes()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name="Disease_Progression")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

baseline = y_train.mean()

y_pred = model.predict(X_test)

coef = model.coef_
feature_means = X_train.mean()

shap_values = pd.DataFrame((X_test - feature_means) * coef, columns=X_test.columns)

shap_values["Baseline"] = baseline
shap_values["Prediction_from_SHAP"] = shap_values.sum(axis=1)
shap_values["Predicted"] = y_pred
shap_values["Actual"] = y_test.values

shap_values["Prediction_Comparison"] = np.where(y_pred > y_test, "Over Prediction",
                                               np.where(y_pred < y_test, "Under Prediction", "Exact"))

print("Intercept:", model.intercept_)
print("Coefficients:", dict(zip(X.columns, coef)))
print("Baseline:", baseline)
print(shap_values)


Intercept: 151.34560453985995
Coefficients: {'age': np.float64(37.904021350074984), 'sex': np.float64(-241.96436231273995), 'bmi': np.float64(542.4287585162899), 'bp': np.float64(347.70384391385636), 's1': np.float64(-931.4888458835163), 's2': np.float64(518.0622769833376), 's3': np.float64(163.41998299131035), 's4': np.float64(275.3179015786484), 's5': np.float64(736.1988589046839), 's6': np.float64(48.67065743196543)}
Baseline: 153.73654390934846
          age        sex        bmi         bp          s1         s2  \
287  1.663955  10.846180  -4.307759  -5.972983 -116.970979  65.277976   
211  3.453897  10.846180  19.077692   7.195035   22.732658  -8.212397   
72   2.352394 -12.218287  -3.138486  -4.775890  -96.464023  25.693757   
321  3.591585  10.846180  27.262599  27.150567  -51.605057  19.366772   
73   0.424765 -12.218287 -11.908030  -1.184613  -36.224840  27.964983   
..        ...        ...        ...        ...         ...        ...   
255  0.011702  10.846180 -36.462754 

# Problem - 4

In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import shap

data = pd.DataFrame({
    "studytime": [2, 3, 1, 4, 2, 3, 1, 4, 3, 2],
    "failures": [0, 1, 0, 2, 1, 0, 3, 0, 1, 2],
    "health": [5, 3, 4, 2, 4, 5, 3, 1, 4, 2],
    "absences": [2, 10, 4, 6, 8, 3, 12, 1, 5, 7],
    "G3": [15, 12, 14, 10, 11, 16, 8, 18, 13, 9]
})

features = data.drop(columns=["G3"])
target = data["G3"]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

baseline = y_train.mean()

y_pred = model.predict(X_test)

explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

shap_df = pd.DataFrame(shap_values.values, columns=X_test.columns)
shap_df["Baseline"] = baseline
shap_df["Prediction_from_SHAP"] = shap_df.sum(axis=1) + baseline
shap_df["Predicted"] = y_pred
shap_df["Actual"] = y_test.values
shap_df["Prediction_Comparison"] = np.where(y_pred > y_test, "Over Prediction",
                                            np.where(y_pred < y_test, "Under Prediction", "Exact"))

print("Intercept:", model.intercept_)
print("Coefficients:", dict(zip(X_train.columns, model.coef_)))
print("Baseline:", baseline)
print(shap_df)


Intercept: 17.537911869789596
Coefficients: {'studytime': np.float64(0.043491685413083916), 'failures': np.float64(-2.555467337126729), 'health': np.float64(-0.6023113228353406), 'absences': np.float64(-0.1342243394645135)}
Baseline: 12.142857142857142
   studytime  failures    health  absences   Baseline  Prediction_from_SHAP  \
0   0.031065  0.365067 -0.602311  0.095875  12.142857             24.175410   
1   0.031065  0.365067 -0.000000 -0.575247  12.142857             24.106599   
2   0.031065  2.920534 -1.204623  0.364323  12.142857             26.397014   

   Predicted  Actual Prediction_Comparison  
0  12.032553      13      Under Prediction  
1  11.963742      12      Under Prediction  
2  14.254157      16      Under Prediction  
